# Analysis & Visualization of Produced Water Chemistry for Environmental & Agricultural Utilization

## SETUP

In [ ]:
# we will all probably need to do this first, so putting this temporary code cell here as a reminder
# once everyone has it installed, this cell should be deleted
!pip install wqchartpy
# if pip install doesn't work for you, go to https://github.com/jyangfsu/WQChartPy/tree/main and follow the "another way" instructions
# if you had to use the "another way" method, you will have to restart your kernal after finishing the install in order for the import to work
#This is my test4 yay
#Sarah comment
#Sarah comment number 2

In [ ]:
# Dependencies
import os
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np
import sklearn
from wqchartpy import triangle_piper

## DATA CLEANING & FILTERING

In [ ]:
# Load the data from the CSV files
df1 = pd.read_csv('../data/split_1_USGSPWDBv2.3n.csv', low_memory=False)
df2 = pd.read_csv('../data/split_2_USGSPWDBv2.3n.csv', low_memory=False)
df3 = pd.read_csv('../data/split_3_USGSPWDBv2.3n.csv', low_memory=False)

# Concatenate the dataframes
frames = [df1, df2, df3]
df_merged = pd.concat(frames, ignore_index=True)

# Save the concatenated dataframe to a new CSV file
df_merged.to_csv('../data/df_merged.csv', index=False)

# List of columns to be removed
columns_to_remove = [
    "IDDB", "SOURCE", "REFERENCE", "LATLONGAPX", "USGSREGION", "BASINCODE", 
    "STATECODE", "COUNTYCODE", "FIELD", "FIELDCODE", "WELLCODE", "TOWNRANGE", 
    "REGDIST", "LOC", "QUAD", "DAY", "DATECOMP", "DATEANALYS", "METHOD", 
    "OPERATOR", "PERMIT", "DFORM", "GROUP", "MEMBER", "AGECODE", "ERA", 
    "PERIOD", "EPOCH", "LAB", "REMARKS", "LITHOLOGY", "POROSITY", "TEMP", 
    "PRESSURE", "SG", "SPGRAV", "SPGRAVT", "RESIS", "RESIST", "PH", "PHT", 
    "EHORP", "COND", "CONDT", "TURBIDITY", "HEM", "MBAS","TDS","TDSCALC", "TSS", "CHARGEBAL", 
    "ACIDITY", "DIC", "DOC", "TOC", "CN", "BOD", "COD", "BENZENE", "TOLUENE", 
    "ETHYLBENZ", "XYLENE", "ACETATE", "BUTYRATE", "FORMATE", "LACTATE", 
    "PHENOLS", "PERC", "PROPIONATE", "PYRUVATE", "VALERATE", "ORGACIDS", 
    "Ar", "CH4", "C2H6", "CO2", "H2", "H2S", "He", "N2", "NH3", "O2", "ALPHA", 
    "BETA", "dD", "H3", "d7Li", "d11B", "d13C", "C14", "d18O", "d34S", 
    "d37Cl", "K40", "d81Br", "Sr87Sr86", "I129", "Rn222", "Ra226", "Ra228", 
    "cull_PH", "cull_MgCa", "cull_KCl", "cull_K5Na", "Ag", "Al", "As", "Au", 
    "B", "BO3", "Be", "Bi", "Cd", "Co", "Cr", "Cs", "Cu", "F", "FeS", "FeAl", 
    "FeAl2O3", "Hg", "I", "Mn", "Mo", "N", "NO2", "NO3", "NO3NO2", "NH4", 
    "TKN", "Ni", "OH", "P", "PO4", "Pb", "Rh", "Rb", "S", "SO3", "HS", "Sb", 
    "Sc", "Se", "Sn", "Ti", "Tl", "U", "V", "W", "Zn"
]

# Remove the specified columns
df_limited_column = df_merged.drop(columns=columns_to_remove, errors='ignore')

# Display the updated merged dataframe
df_limited_column.to_csv('../data/df_limited_column.csv', index=False)

# Remove rows where TDSUSGS <= 35000 (sea water to eliminate all coalebed methane produced water and also the failing analyses)
df_filtered = df_limited_column[df_limited_column['TDSUSGS'] > 35000]

# Save the filtered dataframe to a new CSV file
df_filtered.to_csv('../data/df_filtered_TDS.csv', index=False)


# Fill NaN values in 'KNa', 'K', 'Na', 'Ca', 'Cl', 'SO4', 'Mg' with zeros for calculation
for col in ['KNa', 'K', 'Na', 'Ca', 'Cl', 'SO4', 'Mg']:
    df_filtered[col].fillna(0, inplace=True)

# First, we'll fill NaN values in 'KNa' and 'K' with zeros for the calculation.
df_filtered['KNa'].fillna(0, inplace=True)
df_filtered['K'].fillna(0, inplace=True)

# Apply conditions to calculate 'Na'
# If 'Na' is missing and both 'KNa' and 'K' are present, populate 'Na' with 'KNa' - 'K'
# If 'Na' is missing and 'KNa' is present but 'K' is not, populate 'Na' with 'KNa'

na_mask = df_filtered['Na'].isna()
na_present = df_filtered['Na'] > 0
kna_present = df_filtered['KNa'] > 0
k_present = df_filtered['K'] > 0
k_missing = df_filtered['K'] == 0

df_filtered.loc[k_missing & na_present & kna_present, 'K'] = df_filtered['KNa'] - df_filtered['Na']
df_filtered.loc[na_mask & kna_present & k_present, 'Na'] = df_filtered['KNa'] - df_filtered['K']
df_filtered.loc[na_mask & kna_present & ~k_present, 'Na'] = df_filtered['KNa']

# Remove rows where 'Na' is still missing
df_filtered.dropna(subset=['Na'], inplace=True)

# Remove rows where 'Cl' is missing
df_filtered.dropna(subset=['Cl'], inplace=True)

# Save the updated dataframe 
df_filtered.to_csv('../data/df_filtered_Na_Cl.csv', index=False)

#To calculate the molar concentrations from concentrations given in ppm (parts per million) or mg/L,
#these values need to be converted into moles per liter (M). The formula to convert ppm or mg/L to M is:

                #Molarity (M)=Concentration (mg/L)/ Molar Mass (g/mol) 

#This calculation assumes that 1 ppm is equivalent to 1 mg/L. The molar mass of each element or compound (Na, Ca, Cl, SO4, and Mg) 
#is a constant value based on its atomic or molecular weight.

#Apply the conditions (molar Na > molar Ca) and (molar Cl > molar SO4) and (molar Ca > molar Mg/2) which represent likely unnatural combinations
# Convert concentrations from ppm (mg/L) to Molarity (M)
molar_masses = {'Na': 22.99, 'Ca': 40.08, 'Cl': 35.45, 'SO4': 96.06, 'Mg': 24.305}
for element, molar_mass in molar_masses.items():
    df_filtered[element + '_M'] = df_filtered[element] / molar_mass

# Apply the conditions (molar Na > molar Ca) and (molar Cl > molar SO4) and (molar Ca > molar Mg/2)
condition = (df_filtered['Na_M'] > df_filtered['Ca_M']) & \
            (df_filtered['Cl_M'] > df_filtered['SO4_M']) & \
            (df_filtered['Ca_M'] > df_filtered['Mg_M'] / 2)


df_filtered = df_filtered[condition]

# Save the updated dataframe
df_filtered.to_csv('../data/df_filtered_corrected_elemental_ratios.csv', index=False)

# Filter out rows where USGS charge balance is not between -10 and +10
df_filtered = df_filtered[df_filtered['chargebalance'].between(-10, 10)]

# Save the updated dataframe
df_filtered.to_csv('../data/df_filtered_chargebalance.csv', index=False)

# Calculate charge balance
df_filtered['Cations'] = (df_filtered['Na_M'] * 1) + (df_filtered['Ca_M'] * 2) + (df_filtered['Mg_M'] * 2)
df_filtered['Anions'] = (df_filtered['Cl_M'] * 1) + (df_filtered['SO4_M'] * 2)
df_filtered['CalculatedChargeBalance'] = ((df_filtered['Cations'] - df_filtered['Anions']) / (df_filtered['Cations'] + df_filtered['Anions'])) * 100

# Flag discrepancies between calculated charge balance and existing 'chargebalance' column
threshold = 5  # 5% threshold for discrepancy
df_filtered['ChargeBalanceDiscrepancy'] = abs(df_filtered['CalculatedChargeBalance'] - df_filtered['chargebalance']) > threshold

# Save the updated dataframe with discrepancy flags
df_filtered.to_csv('../data/df_filtered_discrepancy_flags.csv', index=False)

# Calculate DEPTHWELL where it's missing
df_filtered_depth=df_filtered.copy()

for index, row in df_filtered_depth.iterrows():
    if pd.isna(row['DEPTHWELL']):
        if pd.notna(row['DEPTHUPPER']) and pd.notna(row['DEPTHLOWER']):
            # Calculate DEPTHWELL as difference between DEPTHUPPER and DEPTHLOWER
            df_filtered_depth.at[index, 'DEPTHWELL'] = row['DEPTHLOWER'] - row['DEPTHUPPER']
        elif pd.notna(row['DEPTHUPPER']):
            # Populate DEPTHWELL with DEPTHUPPER
            df_filtered_depth.at[index, 'DEPTHWELL'] = row['DEPTHUPPER']
        elif pd.notna(row['DEPTHLOWER']):
            # Populate DEPTHWELL with DEPTHLOWER
            df_filtered_depth.at[index, 'DEPTHWELL'] = row['DEPTHLOWER']

# Remove rows where DEPTHWELL, DEPTHUPPER, and DEPTHLOWER are all missing
df_filtered_depth.dropna(subset=['DEPTHWELL'], inplace=True)

# Save the updated dataframe with updated well depth
df_filtered_depth.to_csv('../data/df_filtered_welldepth.csv', index=False)

## ANALYSIS 

In [ ]:
#Descriptive Statistics

descriptive_stats = df_filtered.describe()
descriptive_stats

In [ ]:
#Descriptive Statistics

specific_columns_stats = df_filtered[['Li','Na', 'Mg','Ca', 'Cl', 'SO4']].describe()
specific_columns_stats

In [ ]:
#Principal Component Analysis (PCA)

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

numerical_columns = df_filtered.select_dtypes(include=['float64', 'int64']).columns

# Check for columns with all NaN values and drop them (if necessary)
df_filtered = df_filtered.dropna(axis=1, how='all')

# Replacing NaN values with the min of the column
df_filtered2=df_filtered.fillna(df_filtered.min())

# Check and handle infinity values
df_filtered2.replace([np.inf, -np.inf], np.nan, inplace=True)
df_filtered2.fillna(df_filtered.mean(), inplace=True)


# Standardize the data: PCA is affected by scale so we need to scale the features in our data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_filtered2[numerical_columns])

# Initialize PCA - uses 2 components for visualization
pca = PCA(n_components=2)
pca_result = pca.fit_transform(scaled_data)

# Create a DataFrame with the PCA results
pca_df = pd.DataFrame(data=pca_result, columns=['PCA1', 'PCA2'])

# Plotting the PCA results
plt.figure(figsize=(8, 6))
plt.scatter(pca_df['PCA1'], pca_df['PCA2'], alpha=0.5)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA of Dataset')
plt.show()

# Display the amount of variance explained by each component
print(f"Explained variance by component: {pca.explained_variance_ratio_}")

In [ ]:
#Cluster Analysis

from sklearn.cluster import KMeans

columns_for_clustering = ['BASIN', 'Li']

basins = df_filtered['BASIN'].unique()
# Create a figure and a set of subplots
fig, axes = plt.subplots(nrows=len(basins), ncols=1, figsize=(10, 5 * len(basins)))

for i, basin in enumerate(basins):
    subset = df_filtered[df_filtered['BASIN'] == basin]
    
# Check if the subset has enough samples for clustering
if len(subset) >= 3:
    kmeans = KMeans(n_clusters=3)
    clusters = kmeans.fit_predict(subset[['Li']])
    df_filtered.loc[df_filtered['BASIN'] == basin, 'cluster'] = clusters

    # Plot each basin's clusters
    ax = axes[i] if len(basins) > 1 else axes
    ax.scatter(subset['Li'], [i]*len(subset), c=clusters, cmap='viridis', label=basin)
    ax.set_title(f'Clusters in {basin} Basin')
    ax.set_xlabel('Lithium Concentration')
    ax.set_yticks([])
    ax.legend()
else:
    # Handle basins with too few samples (e.g., by skipping or plotting without clustering)
    ax = axes[i] if len(basins) > 1 else axes
    ax.scatter(subset['Li'], [i]*len(subset), label=basin)
    ax.set_title(f'{basin} Basin (Insufficient Data for Clustering)')
    ax.set_xlabel('Lithium Concentration')
    ax.set_yticks([])
    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
#Time Series Analysis


import math


# Convert 'DATESAMPLE' to datetime
df_filtered['DATESAMPLE'] = pd.to_datetime(df_filtered['DATESAMPLE'])

# Group by 'WELLNAME' and filter for groups with more than three unique samples
groups = df_filtered.groupby('WELLNAME')
well_groups = {well: group for well, group in groups if group['DATESAMPLE'].nunique() > 3}

# Decide on a reasonable number of plots per figure
plots_per_figure = 10  # Adjust this number based on your preference and screen size
num_wells = len(well_groups)
num_figures = math.ceil(num_wells / plots_per_figure)

# Loop through each subset of wells and create a figure for each
for fig_idx in range(num_figures):
    start_idx = fig_idx * plots_per_figure
    end_idx = start_idx + plots_per_figure
    current_wells = list(well_groups.keys())[start_idx:end_idx]

    # Create figure
    fig, axes = plt.subplots(nrows=min(plots_per_figure, len(current_wells)), ncols=1, figsize=(10, 5 * min(plots_per_figure, len(current_wells))))
    if plots_per_figure > 1:
        axes = axes.flatten()
    else:
        axes = [axes]

    for ax_idx, well in enumerate(current_wells):
        data = well_groups[well].drop_duplicates(subset='DATESAMPLE', keep='first')
        data = data.sort_values(by='DATESAMPLE')

        ax = axes[ax_idx]
        ax.plot(data['DATESAMPLE'], data['Na'], marker='o')
        ax.set_title(f'Time Series for {well}')
        ax.set_xlabel('Sampling Date')
        ax.set_ylabel('Sodium Concentration')
        ax.tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()

In [ ]:
#Test Hypothesis: Li Concentration vs TDS
# Check for NaN values and remove them

import scipy.stats as stats

data = df_filtered[['Li', 'TDSUSGS']].dropna()

# Perform Spearman's rank correlation test
spearman_corr, spearman_pval = stats.spearmanr(data['Li'], data['TDSUSGS'])

print("Spearman's rank correlation test between Li concentration and TDS:")
print("Correlation Coefficient:", spearman_corr)
print("P-value:", spearman_pval)

# Interpret the results
if spearman_pval < 0.05:
    print("There is a statistically significant relationship between Li concentration and TDS.")
else:
    print("There is no statistically significant relationship between Li concentration and TDS.")


In [ ]:
 #Test Hypothesis: Li Concentration vs Depth
    
import scipy.stats as stats
    
# Check for NaN values and remove them
data = df_filtered[['Li', 'DEPTHUPPER']].dropna()

# Perform Spearman's rank correlation test
spearman_corr, spearman_pval = stats.spearmanr(data['Li'], data['DEPTHUPPER'])

print("Spearman's rank correlation test between Li concentration and Depth:")
print("Correlation Coefficient:", spearman_corr)
print("P-value:", spearman_pval)

# Interpret the results
if spearman_pval < 0.05:
    print("There is a statistically significant relationship between Li concentration and Depth.")
else:
    print("There is no statistically significant relationship between Li concentration and Depth.")

In [ ]:
# Regression Analysis

from sklearn.linear_model import LinearRegression

# Remove rows where either Li or DEPTWELL is NaN for regression model training
data_for_regression = df_filtered.dropna(subset=['Li', 'DEPTHWELL'])

# Known Li values
X_known = data_for_regression['Li'] 
y_known = data_for_regression['DEPTHWELL']  

# Rows where Li is unknown
unknown_li = df_filtered[df_filtered['Li'].isna() & df_filtered['DEPTHWELL'].notna()]
X_unknown = unknown_li['Li'] 
y_unknown=unknown_li['DEPTHWELL']

# Check for NaN values in X_unknown and replace with 0
X_unknown.fillna(0, inplace=True)

# Create a linear regression model
model = LinearRegression()

# Fit the model with known Li values
model.fit(X_known.values.reshape(-1, 1), y_known.values)  

# Predict Li for known and unknown DEPTHWELL values
X_pred_known = model.predict(y_known.values.reshape(-1, 1))
X_pred_unknown = model.predict(y_unknown.values.reshape(-1, 1))

# Plotting Li vs DEPTHUPPER with regression line and predicted values
plt.figure(figsize=(10, 6))

# Plot known Li values
plt.scatter(X_known, y_known, color='blue', alpha=0.5, label='Known Li Values')

# Plot predicted Li values for known DEPTH
plt.scatter(X_unknown, y_pred, marker='x', color='red', alpha=0.5, label='Predicted Li on Known Data')

# Plot predicted Li values for unknown DEPTH
#plt.scatter(X_unknown, y_pred_unknown, color='green', alpha=0.5, label='Predicted Li on Unknown Data')

# Plot regression line
li_range = np.linspace(X_known.min(), X_known.max(), 100)
depth_regression = model.predict(li_range.reshape(-1, 1))
plt.plot(li_range, depth_regression, color='black', linestyle='-', label='Regression Line')

plt.xlabel('Li Concentration')
plt.ylabel('Depth')
plt.title('Li Concentration vs Depth with Regression Line')
plt.axvline(0, color='black', linewidth=0.5)
plt.axhline(0, color='black', linewidth=0.5)
plt.grid(color='gray', linestyle='--', linewidth=0.5)
plt.gca().invert_yaxis() 
plt.legend()

# Add regression formula to the plot
plt.text(0.1, 0.9, f'Depth = {model.intercept_:.2f} + {model.coef_[0]:.2f} * Li',
         transform=plt.gca().transAxes, fontsize=12)

## VISUALIZATION

In [ ]:
# Dependencies
from wqchartpy import triangle_piper
from wqchartpy import contour_piper
from wqchartpy import color_piper

    
# The below list of column names is just to document the required column names and column order to feed into wqchartpy - variable is not used; just here for reference
columns = ['Sample','Label','Marker','Size','Color','Alpha','Ca','Mg','Na','K','HCO3','CO3','Cl','SO4']


# Create new dataframe to match format required by wqchartpy for the contour piper plot
df_data_wqchartpyformat = pd.DataFrame()

df_data_wqchartpyformat = pd.DataFrame()
df_data_wqchartpyformat['Sample'] = df_filtered['IDUSGS'].map(str)
df_data_wqchartpyformat['Label'] = 'sample'       # DECISION NEEDED --- we can get fancy with how we want to group this? maybe group it by TDS high/med/low, etc? just have single group for all for now -- only a factor if we use the normal triangle_piper
df_data_wqchartpyformat['Marker'] = 'o'           # DECISION NEEDED --- we can get fancy with how we want to identify markers? maybe group it by TDS high/med/low, etc? just have single shape for all for now -- only a factor if we use the normal triangle_piper
df_data_wqchartpyformat['Color'] = '#FFFF00'      # DECISION NEEDED --- we can get fancy with how we want to color this? maybe group it by TDS high/med/low, etc? just have single color for all for now -- only a factor if we use the normal triangle_piper
df_data_wqchartpyformat['Size'] = 10
df_data_wqchartpyformat['Alpha'] = 0.6

df_data_wqchartpyformat['Ca'] = df_filtered['Ca']
df_data_wqchartpyformat['Mg'] = df_filtered['Mg']
df_data_wqchartpyformat['Na'] = df_filtered['Na']
df_data_wqchartpyformat['K'] = df_filtered['K']
df_data_wqchartpyformat['HCO3'] = df_filtered['HCO3']
df_data_wqchartpyformat['CO3'] = df_filtered['CO3']
df_data_wqchartpyformat['Cl'] = df_filtered['Cl']
df_data_wqchartpyformat['SO4'] = df_filtered['SO4']

# Reset the index
df_data_wqchartpyformat.reset_index(inplace=True, drop=True)

# Show the df
df_data_wqchartpyformat.head(2)

image_name = 'TestPiper1-Normal'
triangle_piper.plot(df_data_wqchartpyformat,unit='mg/L',figname=image_name,figformat='png')
triangle_piper.show()


#image_name = 'TestPiper2-Contour'
#contour_piper.plot(df_data_wqchartpyformat, unit='mg/L', figname=image_name, figformat='png')

#image_file_name = f'{image_name}.png'
#image_folder_name = 'images'
#move_wqchartpy_image_file_to_images_folder(image_file_name, image_folder_name)

#image_name = 'TestPiper3-ColorCoded'
#color_piper.plot(df_data_wqchartpyformat, unit='mg/L', figname=image_name, figformat='png')

#image_file_name = f'{image_name}.png'
#image_folder_name = 'images'
#move_wqchartpy_image_file_to_images_folder(image_file_name, image_folder_name)



In [ ]:
import seaborn as sns

chemicals = ['Na', 'Cl', 'SO4', 'Mg']

# Calculate the correlation matrix using Pearson coefficient
correlation_matrix_pearson = df_filtered[chemicals].corr(method='pearson')

# Alternatively, for Spearman coefficient
# correlation_matrix_spearman = df_filtered[chemicals].corr(method='spearman')

# Plotting the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix_pearson, annot=True, cmap='coolwarm', fmt='.2f', linewidths=.5)
plt.title('Correlation Matrix (Pearson) for Chemical Constituents')
plt.show()